In [325]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [289]:
movies_df=pd.read_csv(r"C:\Users\PMLS\Downloads\tmdb_5000_movies.csv")
credits_df=pd.read_csv(r"C:\Users\PMLS\Downloads\tmdb_5000_credits.csv")
merged_df=pd.merge(movies_df,credits_df,left_on='id',right_on='movie_id')
merged_df.columns
merged_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [290]:
merged_df=merged_df[['movie_id','original_title','overview','genres','keywords','cast','crew']]
merged_df.isna().sum()
merged_df.dropna(inplace=True)
merged_df.shape

(4800, 7)

In [291]:
merged_df.duplicated().sum()

np.int64(0)

In [292]:
def convert(genre):
 gen=[]
 for i in ast.literal_eval(genre):
   gen.append(i['name'])
 return gen

merged_df['genres']=merged_df['genres'].apply(convert)

In [293]:
merged_df['keywords']=merged_df['keywords'].apply(convert)

In [294]:
def convert_cast(cast):
 c=[]
 for i in ast.literal_eval(cast)[0:3]:
   c.append(i['name'])
 return c

merged_df['cast']=merged_df['cast'].apply(convert_cast)

In [295]:
def convert_crew(crew):
 dir=[]
 for i in ast.literal_eval(crew):
   if i['job']=='Director':
     dir.append(i['name'])
 return dir

merged_df['crew']=merged_df['crew'].apply(convert_crew)

In [296]:
merged_df['cast']=merged_df['cast'].apply(lambda x : [i.replace(" ","") for i in x])
merged_df['crew']=merged_df['crew'].apply(lambda x : [i.replace(" ","") for i in x])
merged_df['genres']=merged_df['genres'].apply(lambda x : [i.replace(" ","") for i in x])
merged_df['keywords']=merged_df['cast'].apply(lambda x : [i.replace(" ","") for i in x])
merged_df


,movie_id,original_title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver]","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[JohnnyDepp, OrlandoBloom, KeiraKnightley]","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[DanielCraig, ChristophWaltz, LéaSeydoux]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[ChristianBale, MichaelCaine, GaryOldman]","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[TaylorKitsch, LynnCollins, SamanthaMorton]","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]
...,...,...,...,...,...,...,...
4798,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[Action, Crime, Thriller]","[CarlosGallardo, JaimedeHoyos, PeterMarquardt]","[CarlosGallardo, JaimedeHoyos, PeterMarquardt]",[RobertRodriguez]
4799,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[Comedy, Romance]","[EdwardBurns, KerryBishé, MarshaDietlein]","[EdwardBurns, KerryBishé, MarshaDietlein]",[EdwardBurns]
4800,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[Comedy, Drama, Romance, TVMovie]","[EricMabius, KristinBooth, CrystalLowe]","[EricMabius, KristinBooth, CrystalLowe]",[ScottSmith]
4801,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],"[DanielHenney, ElizaCoupe, BillPaxton]","[DanielHenney, ElizaCoupe, BillPaxton]",[DanielHsia]


In [297]:
merged_df['overview']=merged_df['overview'].apply(lambda x: x.split())
merged_df['tags']=merged_df['overview']+merged_df['genres']+merged_df['keywords']+merged_df['cast']+merged_df['crew']

In [298]:
merged_df=merged_df[['movie_id','original_title','tags']]
merged_df.head()

,movie_id,original_title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [299]:
merged_df['tags']=merged_df['tags'].apply(lambda x: " ".join(x).lower())

C:\Users\PMLS\AppData\Local\Temp\ipykernel_2948\1390321900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['tags']=merged_df['tags'].apply(lambda x: " ".join(x).lower())


In [306]:
def stemmer(text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)

ps=PorterStemmer()
merged_df['tags']=merged_df['tags'].map(stemmer)

C:\Users\PMLS\AppData\Local\Temp\ipykernel_2948\4135501873.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['tags']=merged_df['tags'].map(stemmer)


In [327]:
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(merged_df['tags']).toarray()
cs=cosine_similarity(vectors)
cs

array([[1.        , 0.09837388, 0.06253054, ..., 0.        , 0.        ,
        0.        ],
       [0.09837388, 1.        , 0.06356417, ..., 0.02608203, 0.        ,
        0.02721655],
       [0.06253054, 0.06356417, 1.        , ..., 0.02486824, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02608203, 0.02486824, ..., 1.        , 0.0412393 ,
        0.02129589],
       [0.        , 0.        , 0.        , ..., 0.0412393 , 1.        ,
        0.0860663 ],
       [0.        , 0.02721655, 0.        , ..., 0.02129589, 0.0860663 ,
        1.        ]])

In [394]:


def recommend(movie):
    movie_index = merged_df[merged_df['original_title']==movie].index[0]
    distance=cs[movie_index]
    rc=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    for i in rc:
        print(merged_df.iloc[i[0]].original_title)


In [398]:
recommend("Titanic")

La misma luna
The Switch
Revolutionary Road
What's Eating Gilbert Grape
Opal Dream
